In [6]:
import requests
from bs4 import BeautifulSoup
import os
import cloudscraper
import shutil
import time
import re
from tqdm import tqdm
import pandas as pd

In [7]:
def tile_to_dict(tile):
    item = dict()
    item['title'] = tile.findAll('a')[0]['title']
    item['data-itemid'] = tile['data-itemid']
    item['url'] = tile['data-monetate-producturl']
    try:
        item['colours'] = [a.find('img')['alt'] for a in tile.findAll('li')]
    except Exception as e:
        print(e)
        item['colours'] = 'Exception'
    return item

### Parsing tiles

In [8]:
FOLDER_PERSON = "deep/person"
FOLDER_NO_PERSON = "deep/no_person"
FOLDER_TEST = "deep"
def get_images(url:str, id:str):
    scraper = cloudscraper.create_scraper()
    response = scraper.get(url)
    bs = BeautifulSoup(response.content, 'html.parser')
    imgs = bs.findAll('picture', attrs={'class': 'swiper-zoomable'})
    img_urls = [a['data-highres-images'] for a in imgs]
    for img_url in enumerate(img_urls):
        response = scraper.get(img_url[1])
        filename = f'{id}_{img_url[0]}.jpg'
        if 'alternate' in img_url[1]:
            dst_file = os.path.join(FOLDER_PERSON, filename)
        elif 'lifestyle' in img_url[1]:
            dst_file = os.path.join(FOLDER_NO_PERSON, filename)
        else:
            dst_file = os.path.join(FOLDER_TEST, filename)
        with open(dst_file, "wb") as f:
            f.write(response.content)

In [9]:
def scrape_images(url):
    print('Getting items info')
    tiles = []
    for i in range(999):
        scraper = cloudscraper.create_scraper()
        print(i*32)
        url = url.replace('?', f'?start={i*32}&')
        response = scraper.get(url)
        if response.status_code == 403:
            print('auth error')
            break
        soup = BeautifulSoup(response.content, 'html.parser')
        product_tiles = soup.select('.product-tile')
        t = [tile_to_dict(a) for a in product_tiles]
        if len(t) == 0:
            print('no tiles')
            break
        tiles.extend(t)
    df = pd.DataFrame(tiles)
    df = df.drop_duplicates(subset=['data-itemid'], keep='first').reset_index(drop=True)
    df.to_pickle('df.pkl')
    print('Saving images')
    for a in tqdm(zip(df.url, df['data-itemid']), total=len(df)):
        get_images(a[0], a[1])
    return df

In [10]:
URL = r"https://www.ralphlauren.nl/en/men/clothing/hoodies-sweatshirts/10204?webcat=men%7Cclothing%7Cmen-clothing-hoodies-sweatshirts"
df = scrape_images(URL)
df

Getting items info
0
32
64
96
'NoneType' object is not subscriptable
128
'NoneType' object is not subscriptable
'NoneType' object is not subscriptable
160
192
224
256
288
no tiles
Saving images


100%|██████████| 266/266 [45:06<00:00, 10.18s/it]


,title,data-itemid,url,colours
0,Triple-Pony Plaid-Print Fleece Hoodie,624792,https://www.ralphlauren.nl/en/triple-pony-plai...,[]
1,The RL Fleece Hoodie,490877v1,https://www.ralphlauren.nl/en/the-rl-fleece-ho...,"[Holiday Red, Athletic Green, Harrison Blue]"
2,Polo Ski Fleece Hoodie,624775,https://www.ralphlauren.nl/en/polo-ski-fleece-...,[]
3,Snowflake Estate-Rib Jumper,590613v1,https://www.ralphlauren.nl/en/snowflake-estate...,"[Heritage Snowflake New Fo, Heritage Snowflake..."
4,The RL Fleece Hoodie,568710,https://www.ralphlauren.nl/en/the-rl-fleece-ho...,[]
...,...,...,...,...
261,Colour-Blocked Stretch Jersey Pullover,625356,https://www.ralphlauren.nl/en/colour-blocked-s...,[]
262,Stretch Jersey Half-Zip Pullover,639982,https://www.ralphlauren.nl/en/stretch-jersey-h...,[]
263,Cotton-Blend-Fleece Hoodie,458755,https://www.ralphlauren.nl/en/cotton-blend-fle...,[]
264,Polo Bear Color-Blocked Fleece Hoodie,647816,https://www.ralphlauren.nl/en/polo-bear-color-...,[]
